In [8]:
import numpy
import torch
import scipy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
import TrainTestSplit

ModuleNotFoundError: No module named 'torch'

In [2]:
class Attn(nn.Module):
    def __init__(self, method,feature_size_1,feature_size_2,hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.feature_size_1 = feature_size_1
        self.feature_size_2 = feature_size_2
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.feature_size_1+self.feature_size_2 , hidden_size)
            self.other = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = len(encoder_outputs)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(seq_len)) # B x 1 x S
        if USE_CUDA: attn_energies = attn_energies.cuda()

        # Calculate energies for each encoder output
        for i in range(seq_len):
            attn_energies[i] = self.score(hidden, encoder_outputs[i])

        # Normalize energies to weights in range 0 to 1, resize to 1 x 1 x seq_len
        return F.softmax(attn_energies).unsqueeze(0).unsqueeze(0)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = hidden.dot(encoder_output)
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = hidden.dot(energy)
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = self.other.dot(energy)
            return energy

NameError: name 'nn' is not defined

In [3]:
class FND(nn.Module):
    def __init__(self,method,attn_hidden_size,claim_feature_size,article_term_dim,claim_source_dim,article_source_dim,hidden_size,output_size):
        super(FND,self).__init__()
        self.output_size=output_size
        self.attn=Attn(method,claim_feature_size,article_term_dim,attn_hidden_size)
        self.dense_1=nn.Linear(article_term_dim+claim_source_dim+article_source_dim,hidden_size)
        self.dense_2=nn.Linear(hidden_size,hidden_size)
        self.pred = nn.Linear(hidden_size,output_size)
        
    def forward(self,claim_input,article_inputs,claim_source,article_source):
        attn_wts=self.attn(claim_input,article_inputs)
        claim_specific_article=attn_wts.bmm(article_inputs)
        concatenated_feature = torch.cat((claim_specific_article,torch.tensor(claim_source),torch.tensor(article_source)),1)
        h1 = self.dense_1(concatenated_feature)
        h1 = F.relu(h1)
        h2 = self.dense_2(h2)
        h2 = F.relu(h2)
        pred = self.pred(h2)
        pred = 5*F.sigmoid(pred)
        return pred
        
    
        
        
        
    

NameError: name 'nn' is not defined

In [4]:
X,Y = TrainTestSplit.getBatch('claimCredibility.csv','claimReviews.csv',10)
ATTN_HIDDEN_SIZE = 100
CLAIM_FEATURE_SIZE = len(X[0]['claim'])
ARTICLE_TERM_DIM = len(X[0]['article'][0])
CLAIM_SOURCE_DIM = len(X[0]['claimSource'])
ARTICLE_SOURCE_DIM = len(X[0]['claimSource'])
DENSE_LAYER_DIM = 100
OUTPUT_DIM = 1  #Regression
model = FND('concat',ATTN_HIDDEN_SIZE,CLAIM_FEATURE_SIZE,ARTICLE_TERM_DIM,CLAIM_SOURCE_DIM,ARTICLE_SOURCE_DIM,DENSE_LAYER_DIM,OUTPUT_DIM)
print(model)

NameError: name 'TrainTestSplit' is not defined

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
pred_y=[]
for i in range(len(Y)):
    x=X[i]
    y=Y[i]
    claim_input = x['claim']
    article_inputs = x['article']
    claim_source = x['claimSource']
    article_source = x['articleSource']
    optimizer.zero_grad()
    output = model(claim_input,article_inputs,claim_source,article_source)
    pred_y.append(output)
    loss = loss_fn(output, y)
    print('Loss:'+loss)
    loss.backward()
    optimizer.step()